In [ ]:
#!/usr/bin/python3
import math
import json

# For each context, prior over the states
prior_states = eval(open('irony/synthetic_prior_states.json', 'r').read())
print("prior states: ", prior_states)
# For each state, prior over valence+arousal combination
prior_affect = eval(open('irony/prior_affect.json', 'r').read())

# Prior over QUDs
# hmmm was this ever calculated in the paper?
prior_quds = {
    "state":    0.3,
    "valence":  0.3,
    "arousal":  0.4,
}


states = [1,2,3,4,5]
utterances = [1,2,3,4,5]
quds = [ "state", "valence", "arousal"]
valences = ["positive", "negative"]
arousals = ["high", "low"]
affects = [(v, a) for v in valences for a in arousals]
contexts = list(prior_states.keys())

rationality_factor = 1.0

# the q function in the paper
# 's' is the state of the world, 'A' reps the speaker's affect toward the state
# this serves as a projection from full meaning spact to subset of speaker's interest
def qud(q, s, A):
    if q == "state": return s
    if q == "valence": return A[0]
    if q == "arousal": return A[1]
    print("error")


def literal_listener(s, A, u):
    if s != u:          #if state does not == utterance. This is the basic RSA that returns priors for true states.
        return 0.0
    else:
        return prior_affect[s][A] #so the literal listener will return literally whatever the qud asks for. but what happens if the qud is the state? should we return the probability of a weather state happening? If it doesn't it probs won't change behavior because they're all uniform.


# the U function in the paper, without the log
# this models info gained by listener about topic of interest
# where q is the intended QUD

def exp_utility(u, s, A, q):
    sum = 0.0
    for sp in states:
        for Ap in affects:
            if qud(q, s, A) == qud(q, sp, Ap):
                sum += literal_listener(sp, Ap, u)
    return sum

# the S function in the paper, normalized
# "the speaker S chooses an utterance according to a softmax decision rule"
# in the paper, it says the base is e. How come it's the the utility ftn^ rationality factior here?
def speaker(u, s, A, q):
    norm = 0.0
    for up in utterances:
        norm += math.pow(exp_utility(up, s, A, q), rationality_factor)
    return math.pow(exp_utility(u, s, A, q), rationality_factor) / norm

# the pragmatic L function in the paper, unnormalized
# this is the ftn that models the ambiguity behind QUD, particularly it's held in the sum
def unnorm_pragmatic_listener(s, A, u, context):
    sum = 0.0
    for q in quds:
        sum += prior_quds[q] * speaker(u, s, A, q)
    return prior_states[context][s] * prior_affect[s][A] * sum


# the pragmatic L function in the paper, normalized
def pragmatic_listener(s, A, u, context):
    norm = 0.0
    for sp in states:
        for Ap in affects:
            norm += unnorm_pragmatic_listener(sp, Ap, u, context)
    return unnorm_pragmatic_listener(s, A, u, context) / norm



In [2]:
from collections import defaultdict

output = defaultdict(lambda: defaultdict(lambda: defaultdict(lambda: defaultdict(float))))

def main():
    for context in contexts:
        print("----------------------")
        print("  CONTEXT: %s" % context)
        print("----------------------")
        for u in utterances:
            print("--- utterance: %s ----" % u)
            total_prob = 0.0
            for s in states:
                for A in affects:
                    prob = pragmatic_listener(s, A, u, context)
                    output[context][u][s][A] = prob
                    print("  s: %s, A: %s:\t%f" % (s, A, prob))
                    total_prob += prob
            print("  total_prob: %f" % total_prob)
main()

----------------------
  CONTEXT: WC5
----------------------
--- utterance: 1 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.779717
  s: 1, A: ('negative', 'low'):	0.132183
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.000009
  s: 2, A: ('negative', 'low'):	0.000005
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.000238
  s: 3, A: ('negative', 'high'):	0.001196
  s: 3, A: ('negative', 'low'):	0.008721
  s: 4, A: ('positive', 'high'):	0.000084
  s: 4, A: ('positive', 'low'):	0.000040
  s: 4, A: ('negative', 'high'):	0.000021
  s: 4, A: ('negative', 'low'):	0.000090
  s: 5, A: ('positive', 'high'):	0.068969
  s: 5, A: ('positive', 'low'):	0.004026
  s: 5, A: ('negative', 'high'):	0.004168
  s: 5, A: ('negative', 'low'):	0.000533
  total_prob: 1.000000
--- utterance: 2 ----
  s: 1, A: ('positive', 'high'):	0.000000


  s: 4, A: ('positive', 'high'):	0.000009
  s: 4, A: ('positive', 'low'):	0.000015
  s: 4, A: ('negative', 'high'):	0.000001
  s: 4, A: ('negative', 'low'):	0.000001
  s: 5, A: ('positive', 'high'):	0.310188
  s: 5, A: ('positive', 'low'):	0.080395
  s: 5, A: ('negative', 'high'):	0.008251
  s: 5, A: ('negative', 'low'):	0.001607
  total_prob: 1.000000
----------------------
  CONTEXT: WC85
----------------------
--- utterance: 1 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.000001
  s: 1, A: ('negative', 'low'):	0.000000
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.568289
  s: 2, A: ('negative', 'low'):	0.351081
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.001042
  s: 3, A: ('negative', 'high'):	0.005237
  s: 3, A: ('negative', 'low'):	0.038202
  s: 4, A: ('positive', 'high'):	0.011568
  s: 4, A: ('positive

  s: 5, A: ('positive', 'high'):	0.098401
  s: 5, A: ('positive', 'low'):	0.032145
  s: 5, A: ('negative', 'high'):	0.007877
  s: 5, A: ('negative', 'low'):	0.002020
  total_prob: 1.000000
--- utterance: 3 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.149692
  s: 1, A: ('negative', 'low'):	0.077452
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.017238
  s: 2, A: ('negative', 'low'):	0.049908
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.013794
  s: 3, A: ('negative', 'high'):	0.006768
  s: 3, A: ('negative', 'low'):	0.128575
  s: 4, A: ('positive', 'high'):	0.017058
  s: 4, A: ('positive', 'low'):	0.246814
  s: 4, A: ('negative', 'high'):	0.008757
  s: 4, A: ('negative', 'low'):	0.175059
  s: 5, A: ('positive', 'high'):	0.036441
  s: 5, A: ('positive', 'low'):	0.065188
  s: 5, A: ('negative', 'high'):	0.004535


  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.001884
  s: 2, A: ('negative', 'low'):	0.000650
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.009620
  s: 3, A: ('negative', 'high'):	0.003731
  s: 3, A: ('negative', 'low'):	0.015199
  s: 4, A: ('positive', 'high'):	0.113705
  s: 4, A: ('positive', 'low'):	0.191472
  s: 4, A: ('negative', 'high'):	0.007797
  s: 4, A: ('negative', 'low'):	0.018575
  s: 5, A: ('positive', 'high'):	0.000499
  s: 5, A: ('positive', 'low'):	0.000129
  s: 5, A: ('negative', 'high'):	0.000013
  s: 5, A: ('negative', 'low'):	0.000003
  total_prob: 1.000000
----------------------
  CONTEXT: WC29
----------------------
--- utterance: 1 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.379420
  s: 1, A: ('negative', 'low'):	0.064322
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive

  s: 4, A: ('positive', 'low'):	0.008918
  s: 4, A: ('negative', 'high'):	0.004707
  s: 4, A: ('negative', 'low'):	0.020080
  s: 5, A: ('positive', 'high'):	0.060181
  s: 5, A: ('positive', 'low'):	0.003513
  s: 5, A: ('negative', 'high'):	0.003637
  s: 5, A: ('negative', 'low'):	0.000465
  total_prob: 1.000000
--- utterance: 2 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.087585
  s: 1, A: ('negative', 'low'):	0.019379
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.307468
  s: 2, A: ('negative', 'low'):	0.374562
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.006288
  s: 3, A: ('negative', 'high'):	0.007484
  s: 3, A: ('negative', 'low'):	0.109403
  s: 4, A: ('positive', 'high'):	0.007743
  s: 4, A: ('positive', 'low'):	0.020459
  s: 4, A: ('negative', 'high'):	0.002557
  s: 4, A: ('negative', 'low'):	0.021856
 

...

  s: 3, A: ('positive', 'low'):	0.028717
  s: 3, A: ('negative', 'high'):	0.014091
  s: 3, A: ('negative', 'low'):	0.267674
  s: 4, A: ('positive', 'high'):	0.020977
  s: 4, A: ('positive', 'low'):	0.303508
  s: 4, A: ('negative', 'high'):	0.010769
  s: 4, A: ('negative', 'low'):	0.215271
  s: 5, A: ('positive', 'high'):	0.000000
  s: 5, A: ('positive', 'low'):	0.000000
  s: 5, A: ('negative', 'high'):	0.000000
  s: 5, A: ('negative', 'low'):	0.000000
  total_prob: 1.000000
--- utterance: 4 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.000147
  s: 1, A: ('negative', 'low'):	0.000067
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.012535
  s: 2, A: ('negative', 'low'):	0.032012
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.007281
  s: 3, A: ('negative', 'high'):	0.001184
  s: 3, A: ('negative', 'low'):	0.035699
 

  s: 2, A: ('negative', 'high'):	0.096119
  s: 2, A: ('negative', 'low'):	0.033166
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.005438
  s: 3, A: ('negative', 'high'):	0.002109
  s: 3, A: ('negative', 'low'):	0.008592
  s: 4, A: ('positive', 'high'):	0.032353
  s: 4, A: ('positive', 'low'):	0.054480
  s: 4, A: ('negative', 'high'):	0.002218
  s: 4, A: ('negative', 'low'):	0.005285
  s: 5, A: ('positive', 'high'):	0.523731
  s: 5, A: ('positive', 'low'):	0.135742
  s: 5, A: ('negative', 'high'):	0.013931
  s: 5, A: ('negative', 'low'):	0.002714
  total_prob: 1.000000
----------------------
  CONTEXT: WC71
----------------------
--- utterance: 1 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.185591
  s: 1, A: ('negative', 'low'):	0.031463
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.136257
  s: 2, A: ('negative

  s: 4, A: ('negative', 'low'):	0.003989
  s: 5, A: ('positive', 'high'):	0.011032
  s: 5, A: ('positive', 'low'):	0.000644
  s: 5, A: ('negative', 'high'):	0.000667
  s: 5, A: ('negative', 'low'):	0.000085
  total_prob: 1.000000
--- utterance: 2 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.561561
  s: 1, A: ('negative', 'low'):	0.124250
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.054945
  s: 2, A: ('negative', 'low'):	0.066935
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.007210
  s: 3, A: ('negative', 'high'):	0.008581
  s: 3, A: ('negative', 'low'):	0.125445
  s: 4, A: ('positive', 'high'):	0.004646
  s: 4, A: ('positive', 'low'):	0.012276
  s: 4, A: ('negative', 'high'):	0.001534
  s: 4, A: ('negative', 'low'):	0.013115
  s: 5, A: ('positive', 'high'):	0.013664
  s: 5, A: ('positive', 'low'):	0.004464
 

  s: 3, A: ('positive', 'low'):	0.000002
  s: 3, A: ('negative', 'high'):	0.000000
  s: 3, A: ('negative', 'low'):	0.000011
  s: 4, A: ('positive', 'high'):	0.009338
  s: 4, A: ('positive', 'low'):	0.027130
  s: 4, A: ('negative', 'high'):	0.001116
  s: 4, A: ('negative', 'low'):	0.011195
  s: 5, A: ('positive', 'high'):	0.000000
  s: 5, A: ('positive', 'low'):	0.000000
  s: 5, A: ('negative', 'high'):	0.000000
  s: 5, A: ('negative', 'low'):	0.000000
  total_prob: 1.000000
--- utterance: 5 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.892576
  s: 1, A: ('negative', 'low'):	0.055041
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.029444
  s: 2, A: ('negative', 'low'):	0.010160
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.000001
  s: 3, A: ('negative', 'high'):	0.000001
  s: 3, A: ('negative', 'low'):	0.000002
 

  s: 4, A: ('negative', 'high'):	0.008186
  s: 4, A: ('negative', 'low'):	0.069978
  s: 5, A: ('positive', 'high'):	0.067799
  s: 5, A: ('positive', 'low'):	0.022148
  s: 5, A: ('negative', 'high'):	0.005427
  s: 5, A: ('negative', 'low'):	0.001392
  total_prob: 1.000000
--- utterance: 3 ----
  s: 1, A: ('positive', 'high'):	0.000000
  s: 1, A: ('positive', 'low'):	0.000000
  s: 1, A: ('negative', 'high'):	0.176499
  s: 1, A: ('negative', 'low'):	0.091322
  s: 2, A: ('positive', 'high'):	0.000000
  s: 2, A: ('positive', 'low'):	0.000000
  s: 2, A: ('negative', 'high'):	0.019471
  s: 2, A: ('negative', 'low'):	0.056372
  s: 3, A: ('positive', 'high'):	0.000000
  s: 3, A: ('positive', 'low'):	0.033186
  s: 3, A: ('negative', 'high'):	0.016284
  s: 3, A: ('negative', 'low'):	0.309335
  s: 4, A: ('positive', 'high'):	0.008645
  s: 4, A: ('positive', 'low'):	0.125085
  s: 4, A: ('negative', 'high'):	0.004438
  s: 4, A: ('negative', 'low'):	0.088720
  s: 5, A: ('positive', 'high'):	0.023643


In [3]:
#example on marginalizing over the affect to get the probability of a state. This is done over every utterance/context combination
# for c in range(len(contexts)):
#     for u in output[contexts[c]].keys():
#         out = output[contexts[c]][u]
#         y = [sum(affect.values()) for affect in out.values()]
#         x = list(out.keys())

In [4]:
import matplotlib.pyplot as plt
import numpy as np

fig = plt.figure(figsize=(10, 10))
labels = ["terrible", "bad", "neutral", "good", "amazing"]


for c in range(len(contexts)):
    for u in output[contexts[c]].keys():
        out = output[contexts[c]][u]
        y = [sum(affect.values()) for affect in out.values()]
        x = list(out.keys())

        ax1 = plt.subplot2grid((len(contexts),5),(c,u-1))
        ax1.set_ylim(0, 1)
        ax1.plot(list(out.keys()),y)
        plt.grid()
        plt.xticks(list(out.keys()), labels) 
        plt.yticks(np.arange(0.0, 1.0, 0.25))


fig.subplots_adjust(hspace=.5, wspace=.5)
plt.show()

<Figure size 1000x1000 with 500 Axes>

In [5]:
#write output 
def ddict2dict(d):
    '''
    convert recursive defaultdict to dict
    '''
    for k, v in d.items():
        if isinstance(v, dict):
            d[k] = ddict2dict(v)
    return dict(d)

# for c in range(len(contexts)):
#     for u in output[contexts[c]].keys():
#         out = output[contexts[c]][u]
#         y = [sum(affect.values()) for affect in out.values()]
#         x = list(out.keys())

for c in range(len(contexts)):
    for u in output[contexts[c]].keys():
        out = output[contexts[c]][u]
        print([sum(affect.values()) for affect in out.values()])
        output[contexts[c]][u] = [sum(affect.values()) for affect in out.values()]
        
output = ddict2dict(output)
with open('synthetic_fig_5_labels.json', 'w') as fp:
    fp.write(str(output))

[0.9119004953190482, 1.3880507201498983e-05, 0.010154723203147392, 0.0002345305251884124, 0.07769637044541451]
[0.8154757198172514, 0.00010242268826075833, 0.0384359376063651, 0.0008438272094407498, 0.14514209267868206]
[0.7100654403856603, 4.5746941775119904e-05, 0.15145506778445453, 0.0014690586299006497, 0.1369646862582092]
[0.5222998950454614, 3.198662277910022e-05, 0.04678745318454647, 0.005279906283604271, 0.4256007588636087]
[0.35328723915201454, 1.1233495120203604e-05, 0.008906468593585789, 0.0008689785385135444, 0.636926080220766]
[0.3685605465535122, 0.571799022502227, 0.009608209984892476, 7.334188449533845e-06, 0.05002488677091852]
[0.07044493534509147, 0.9018028255114452, 0.007773009958271484, 5.6400578995232346e-06, 0.01997358912729229]
[0.1194260925369484, 0.7842230354260274, 0.05963451085668377, 1.911750767514853e-05, 0.03669724367266543]
[0.11427776934437522, 0.7133240814414105, 0.023965374823143724, 8.93839055962073e-05, 0.1483433904854741]
[0.1394292073463958, 0.4518

In [6]:
prior_affect

{1: {('negative', 'high'): 0.8225806451612904,
  ('negative', 'low'): 0.1774193548387097,
  ('positive', 'high'): 0.0,
  ('positive', 'low'): 0.0},
 2: {('negative', 'high'): 0.453125,
  ('negative', 'low'): 0.546875,
  ('positive', 'high'): 0.0,
  ('positive', 'low'): 0.0},
 3: {('negative', 'high'): 0.058823529411764705,
  ('negative', 'low'): 0.8382352941176471,
  ('positive', 'high'): 0.0,
  ('positive', 'low'): 0.10294117647058823},
 4: {('negative', 'high'): 0.030612244897959183,
  ('negative', 'low'): 0.25510204081632654,
  ('positive', 'high'): 0.20408163265306123,
  ('positive', 'low'): 0.5102040816326531},
 5: {('negative', 'high'): 0.026845637583892617,
  ('negative', 'low'): 0.006711409395973154,
  ('positive', 'high'): 0.738255033557047,
  ('positive', 'low'): 0.22818791946308725}}